In [89]:
%load_ext autoreload
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import re
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [92]:
pd.set_option('display.max_rows', 100)
SEED = 42
USE_nG = True # creting data only from nG subset

In [93]:
# https://docs.google.com/spreadsheets/d/1MDleLmQ0Nlcg62x95e3xnkc5_j_i4IK_KQEHccDosG8/edit?usp=sharing
sheet_id = "1MDleLmQ0Nlcg62x95e3xnkc5_j_i4IK_KQEHccDosG8"
sheet_name = "clean"
url = f"https://docs.google.com/spreadsheets/d/{sheet_id}/gviz/tq?tqx=out:csv&sheet={sheet_name}"

In [112]:
df = pd.read_csv(url, 
                 header=0,
                 usecols=['patient', 'is_good', 'localization', 'comments'],
                 index_col=None, dtype={'patient':str,
                                        'is_good':str,
                                        'localization':str,
                                        'comments':str})

if USE_nG:
    df = df[df['patient'].apply(lambda x: x[0]=='n' or x[0]=='G')]
df_good = df.query('is_good == "1"')

In [113]:
df_good['localization'].value_counts()

left front              29
right front             28
left tempor             18
right tempor            15
left occipit             2
right tempor occipit     1
left tempor occipit      1
left occipit tempor      1
left front tempor        1
right occipit            1
Name: localization, dtype: int64

In [114]:
manual_mapping = {'right tempor occipit':'occipit',
                  'left tempor occipit':'occipit',
                  'left occipit tempor':'occipit',
                  'left occipit':'occipit',
                  'right occipit':'occipit',
                  'left front tempor':'left tempor'
                 }

for k,v in manual_mapping.items():
    mask = df_good['localization'] == k
    df_good.loc[mask, 'localization'] = v

<ipython-input-114-deea09b74b72>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_good.loc[mask, 'localization'] = v


In [115]:
df_good['localization'].value_counts()

left front      29
right front     28
left tempor     19
right tempor    15
occipit          6
Name: localization, dtype: int64

In [116]:
df_train, df_test = train_test_split(df_good, 
                                     stratify=df_good['localization'],
                                     random_state=SEED,
                                     test_size=10)

In [117]:
df_train.shape, df_test.shape

((87, 4), (10, 4))

In [118]:
metadata = {'train':df_train['patient'].values,
            'test':df_test['patient'].values}
np.save('metadata/metadata_fcd_nG.npy', metadata)